In [ ]:
import pandas as pd
import numpy as np 
from matplotlib import style, pyplot as plt
import seaborn as sns

print('Imported libraries')

In [ ]:
df = pd.read_csv('../input/currency-exchange-rates/forex.csv', index_col='date')

df.index = pd.to_datetime(df.index)

print('Data has '+str(len(df[df.slug=='USD/BRL']))+' observations per exchange rate')

In [ ]:
style.use('seaborn-darkgrid')

def currency_plotter(slug):
    plt.figure(figsize=(15, 8))
    plt.plot(df[df.slug==slug].close,'g',  linewidth=0.8,label=slug+' Close')
    plt.title(slug)
    plt.legend()
    plt.show()
    
currency_plotter('USD/BRL')
currency_plotter('GBP/BRL')

In [ ]:
# For small nominal value currencies, especially made to check brazilian case. Might not work with other countries

def country_plotter(country_coin_name, country_name):
    plt.figure(figsize=(15, 8))        
    plt.title(country_name)
    country_df = df[df.currency==country_coin_name]
    for slug in country_df.slug.unique():
        if country_df[country_df.slug==slug].close.mean() > 0.5:
            plt.plot(country_df[country_df.slug==slug].close, linewidth=0.8, label=slug)
        else:
            plt.plot(country_df[country_df.slug==slug].close*100, linewidth=0.8, label=(slug+' *10^2'))

    plt.legend()
    plt.show()
    return country_df
    
bra_df = country_plotter('BRL', 'Brazil')

In [ ]:
country_plotter('ARS', 'Argentina')

# **Notice that a sudden change in the comparative value of a currency might indicate some monetary policy addopted by the country**
# 
The plot bellow shows a devaluation of the egyptian currency in order to meet IMF demands.

https://www.theguardian.com/world/2016/nov/03/egypt-devalues-currency-meet-imf-demands-loan

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTHKI5tj3wG3UEi9yjMzFh9jAAu0_oZ8QsKkw&usqp=CAU">


In [ ]:
country_plotter('EGP', 'Egypt')

# Simple returns for BRL

In [ ]:
def variation_plot(country_df):
    for slug in country_df.slug.unique():
        plt.figure(figsize=(10,5))
        plt.title(slug)
        plt.plot((country_df[country_df.slug==slug].close/country_df[country_df.slug==slug].close.shift(1))-1, linewidth=0.8)
    plt.show()

variation_plot(bra_df)

# Logarithmic returns for BRL


In [ ]:
def log_variation_plot(country_df):
    for slug in country_df.slug.unique():
        plt.figure(figsize=(10,5))
        plt.title(slug)
        plt.plot(np.log(country_df[country_df.slug==slug].close/country_df[country_df.slug==slug].close.shift(1)), linewidth=0.8)
    plt.show()
log_variation_plot(bra_df)

IGP-M is a more accurate measure when talking about wholesales price, and it should be more impacted by variations in the exchange rate, especially the USD/BRL.



In [ ]:
print(bra_df[bra_df.slug=='GBP/BRL'].index.min())

print(bra_df[(bra_df.slug=='USD/BRL')].index.max())


In [ ]:
ipca1 = pd.read_csv('../input/ipca-brazil-1/ipca - ipca.csv', index_col='date')
ipca1.index = pd.to_datetime(ipca1.index)
ipca1


dummie_list_gbp = []
for date in ipca1.index:
    dummie_list_gbp.append(bra_df[(bra_df.index.month == date.month) & (bra_df.index.year == date.year) & (bra_df.slug == 'GBP/BRL')].close.mean())
    
dummie_series = pd.DataFrame()
dummie_series['exchange_gbp'] = pd.Series(dummie_list_gbp)
dummie_series['variations'] = (dummie_series['exchange_gbp']/dummie_series['exchange_gbp'].shift(1) -1)*100
dummie_series = dummie_series.set_index(ipca1.index)
plt.figure(figsize=(15,8))
plt.plot(dummie_series['variations'], linewidth=0.8,label = 'Exchange Rate Month Variations')
plt.plot(ipca1['value']*10, linewidth=0.8,label = 'Official Inflation Rate Month Variations x10')
plt.title('GBP/BRL X Inflation Rate')
plt.legend()
plt.show()

In [ ]:
igpm = pd.read_csv('../input/igpm-brasil/igpm - igpm (1).csv', index_col='date')
igpm.index = pd.to_datetime(igpm.index)
igpm = igpm.drop(columns=['garbage1', 'garbage2', 'garbage3'])

dummie_list_usd = [None, None]
for date in ipca1.index:
    dummie_list_usd.append(bra_df[(bra_df.index.month == date.month) & (bra_df.index.year == date.year) & (bra_df.slug == 'USD/BRL')].close.mean())
    
dummie_series2 = pd.DataFrame()
dummie_series2['exchange_usd'] = pd.Series(dummie_list_usd)
dummie_series2['variations'] = (dummie_series2['exchange_usd']/dummie_series2['exchange_usd'].shift(1) -1)*100
dummie_series2 = dummie_series2.set_index(igpm.index)
plt.figure(figsize=(15,8))
plt.plot(dummie_series2['variations'], linewidth=0.8,label = 'Exchange Rate Month Variations')
plt.plot(igpm['value']*10, linewidth=0.8,label = 'Wholesome Inflation Rate Month Variations*10')
plt.title('USD/BRL X IGPM')
plt.legend()
plt.show()

In [ ]:
# Scatter

plt.figure(figsize=(15,8))
sns.regplot(x=dummie_series2['exchange_usd'], y=igpm['value'], scatter_kws={'s':10})
plt.xlabel('USD/BRL')
plt.ylabel('IGP-M')
plt.title('USD/BRL X IGPM')
plt.legend()
plt.show()

Inflation can be partly explained by exchange rates, as we can see on the graphs above. 
It's also interesting to work on the causation relations between exchange rates and the brazilian trade balance.


In [ ]:
trade_balance = pd.read_csv('../input/tradebalance-brazil/STP-20210430230456140 - STP-20210430230456140 (1).csv', index_col='date')
trade_balance = trade_balance.drop(columns=['garbage1', 'garbage2', 'garbage3'])
trade_balance.index = pd.to_datetime(trade_balance.index)

dummie_list_trade = []
for date in trade_balance.index:
    dummie_list_trade.append(bra_df[(bra_df.index.month == date.month) & (bra_df.index.year == date.year) & (bra_df.slug == 'USD/BRL')].close.mean())
    
dummie_series3 = pd.DataFrame()
dummie_series3['exchange_usd'] = pd.Series(dummie_list_trade)
dummie_series3['variations'] = (dummie_series3['exchange_usd']/dummie_series3['exchange_usd'].shift(1) -1)*100
dummie_series3 = dummie_series3.set_index(trade_balance.index)
plt.figure(figsize=(15,8))
plt.plot(dummie_series3['exchange_usd'], linewidth=0.8,label = 'Exchange Rate Month Average')
print(trade_balance['value'])
plt.plot(trade_balance['value']/1000, linewidth=0.8,label = 'Trade Balance * 10^-3')
plt.title('Trade Balance X USD/BRL')
plt.legend()
plt.show()

In [ ]:
fiscal = pd.read_csv('../input/debt-gdp-ratio/STP-20210501170518189 - STP-20210501170518189.csv', index_col='date')
fiscal = fiscal.drop(columns=['garbage1', 'garbage2', 'garbage3'])
fiscal.index = pd.to_datetime(fiscal.index)

dummie_list_fiscal = []
for date in fiscal.index:
    dummie_list_fiscal.append(bra_df[(bra_df.index.month == date.month) & (bra_df.index.year == date.year) & (bra_df.slug == 'USD/BRL')].close.mean())

dummie_series4 = pd.DataFrame()
dummie_series4['exchange_usd'] = pd.Series(dummie_list_fiscal)
dummie_series4['variations'] = (dummie_series4['exchange_usd']/dummie_series4['exchange_usd'].shift(1) -1)*100
dummie_series4 = dummie_series4.set_index(fiscal.index)
plt.figure(figsize=(15,8))
plt.plot(dummie_series4['exchange_usd']*10, linewidth=0.8,label = 'Exchange Rate Month Average x10')
plt.plot(fiscal['value'][:-1].astype(float), linewidth=0.8,label = '% of GDP committed to public expenses ')
plt.title('% of GDP committed to public expenses X USD/BRL x10')
plt.legend()
plt.show()

In [ ]:
fiscal['variations'] = (fiscal['value'][:-1].astype(float)/fiscal['value'][:-1].astype(float).shift(1) -1)*100
plt.figure(figsize=(15,8))
plt.plot(fiscal['variations'], linewidth=0.8,label = '')
plt.plot(dummie_series4['variations'], linewidth=0.8,label = 'Monthly Variation of Exchange Rate Month Average')
plt.title('Monthly variation of % of GDP committed to public expenses X Variation of USD/BRL')
plt.legend()
plt.show()

EXPLORE THE RESULTS OF PREVIOUS GRAPHS


In [ ]:
selic = pd.read_csv('../input/brazil-selic1/STP-20210505195738996 - STP-20210505195738996.csv', index_col='date')
selic = selic.drop(columns=['garbage1', 'garbage2', 'garbage3'])
selic.index = pd.to_datetime(selic.index)

dummie_list_selic = []
for date in selic.index:
    dummie_list_selic.append(bra_df[(bra_df.index.month == date.month) & (bra_df.index.year == date.year) & (bra_df.slug == 'USD/BRL')].close.mean())

dummie_series5 = pd.DataFrame()
dummie_series5['exchange_usd'] = pd.Series(dummie_list_selic)
dummie_series5['variations'] = (dummie_series5['exchange_usd']/dummie_series5['exchange_usd'].shift(1) -1)*100
dummie_series5 = dummie_series5.set_index(selic.index)
plt.figure(figsize=(15,8))
plt.plot(dummie_series5['exchange_usd']*5, linewidth=0.8,label = 'Exchange Rate Month Average * 5')
plt.plot(selic['value'], linewidth=0.8,label = "Monthly accumulated interest rates")
plt.title('Interest Rates X USD/BRL')
plt.legend()
plt.show()

In [ ]:
selic['variations'] = (selic['value']/selic['value'].shift(1) -1)*100
plt.figure(figsize=(15,8))
plt.plot(selic['variations'], linewidth=0.8,label = 'Monthly variation of SELIC rate')
plt.plot(dummie_series5['variations'], linewidth=0.8,label = 'Monthly Variation of Exchange Rate Month Average')
plt.title('Variation of Interest Rates X Variation of USD/BRL')
plt.legend()
plt.show()